<a href="https://colab.research.google.com/github/evgeny-kolonsky/Lab3_MW/blob/main/Lab3_MW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lab 3  Mechanical Waves

Ebgeny Kolonsky 2024

v.0.1.2

In [3]:
import numpy as np
from scipy import linalg as LA
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML

%matplotlib notebook

https://lpsa.swarthmore.edu/MtrxVibe/MatrixAll.html

https://cooperrc.github.io/computational-mechanics/module_05/03_Good_Vibrations.html#./02_Keep_it_steady.ipynb


Model equations

$$
M_i \ddot{\theta_i}  -k\theta_{i-1} + 2 k \theta_i - k \theta_{i+1} = 0, \quad i = 1 \dots n-1
$$

Boundary conditions

Forced harmonic oscillation
$$
\theta_0(t) = \eta_0 \, e^{i \omega t}
$$

The other end is fixed
$$
\theta_n = 0
$$

Coordinates vector
$$
\Theta =  \begin{bmatrix}
      \theta_1 \\
      \vdots\\
      \theta_n \end{bmatrix} ,
$$

Inertia matrix
$$
M =  \begin{bmatrix}
      M_1 &     \\
       &\ddots & \\
       &            & M_n
      \end{bmatrix} ,
$$
Elastity matrix
$$
K =  \begin{bmatrix}
      2 k &  -k  &  &         \\
      -k  &  2k  & -k &          \\
         & -k   & 2k  & -k &         \\
      & & &\ddots & -k\\
      & &         & -k & 2 k
      \end{bmatrix}
$$

External forces vector
$$
F =  \begin{bmatrix}
      \eta_0 k \\
      \vdots\\
      0 \end{bmatrix} ,
$$

Motion equitaion in matrix form
$$
I \ddot \Theta + K \Theta = F \, e^{i\omega t}
$$

$$
Z = M^{-1}  K
$$
Eigenvalues $\lambda$ are solution of equation
$$
\det |Z - \lambda I| = 0
$$

Eigenfrequencies are squares of eigenvalues
$$
\omega_i^2 = -\lambda_i
$$

$$
\Theta(t) = \sum A_k E_k e^{i \omega_k t}
$$

$$
\Theta(0) = A E, \quad A = \Theta(0) E^{-1}
$$


Constants and parameters

In [118]:
# single rod
l = 460e-3 # length of rod
m = 42e-3 # mass of rod
J = m * l**2 / 12 # moment of inertia of rod

# the mechanical wave system
L = 720-3 # length of the system
n = 72 # number of elements in rotational oscillations system
d = 10e-3 # distance between rods

k = 1 # elastity coeff

v = d * np.sqrt(k/J)
print(f'wave velocity expected {v=} m/s')

wave velocity expected v=0.3674583716210941 m/s


In [119]:
K = k * np.diag(np.ones(n) * 2, 0) + \
    np.diag(-np.ones(n-1)   , 1) +  \
    np.diag(-np.ones(n-1)   ,-1)

M = np.diag(np.ones(n) * J)

Z = LA.inv(M) @ K

X0 = np.zeros(n)
X0[0] = 1

lambdas, E = LA.eig(Z)
isort = np.argsort(lambdas.real)
lambdas = lambdas[isort]
E = E[:,isort]
omega = np.sqrt(lambdas)

A = X0 @ LA.inv(E) # amplitudes


In [106]:
np.real(omega/2/np.pi)

array([24.81343382, 24.81725983, 24.82362733, 24.83252259, 24.84392643,
       24.85781429, 24.87415632, 24.89291746, 24.91405754, 24.93753143,
       24.96328913, 24.99127596, 25.02143271, 25.05369581, 25.0879975 ,
       25.12426606, 25.16242597, 25.20239813, 25.24410011, 25.28744631,
       25.33234825, 25.37871473, 25.42645213, 25.47546461, 25.5256543 ,
       25.57692163, 25.62916549, 25.68228346, 25.73617208, 25.79072705,
       25.84584344, 25.90141594, 25.95733904, 26.01350725, 26.06981533,
       26.12615843, 26.18243233, 26.23853361, 26.29435981, 26.34980963,
       26.40478309, 26.45918166, 26.51290847, 26.5658684 , 26.61796825,
       26.66911685, 26.71922523, 26.7682067 , 26.81597696, 26.86245427,
       26.90755946, 26.95121611, 26.9933506 , 27.03389219, 27.07277315,
       27.10992875, 27.14529742, 27.17882077, 27.21044365, 27.24011422,
       27.26778398, 27.29340787, 27.31694426, 27.338355  , 27.35760548,
       27.37466465, 27.38950503, 27.40210278, 27.41243767, 27.42

In [105]:
def X(t):
    return   A * np.exp(1j * omega * t - t/2/tau) @ E

In [122]:
fig, ax = plt.subplots()
x = range(n)
tau = 5 # s - decay time
t_max = tau * 2 # s
delta_t = 0.2 # s
N = int(t_max / delta_t)
t = np.linspace(0, t_max, N)
y = X(0)
bottom = 0
h_stem = ax.stem(x, y, bottom=bottom,  linefmt='-.')

def update(i):
    y = X(t[i])

    # markerline
    h_stem[0].set_ydata(y)
    h_stem[0].set_xdata(x)  # not necessary for constant x

    # stemlines
    h_stem[1].set_paths([np.array([[xx, bottom],
                                   [xx, yy]]) for (xx, yy) in zip(x, y)])

    # baseline
    h_stem[2].set_xdata([np.min(x), np.max(x)])
    h_stem[2].set_ydata([bottom, bottom])  # not necessary for constant bottom

anim = FuncAnimation(fig, update, frames=range(1, N, 1), interval=delta_t)
anim.save('so.gif', dpi=80)
#HTML(anim.to_jshtml())

<IPython.core.display.Javascript object>

/usr/local/lib/python3.10/dist-packages/matplotlib/cbook.py:1699: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:3387: ComplexWarning: Casting complex values to real discards the imaginary part
  _data[indx] = dval
/usr/local/lib/python3.10/dist-packages/matplotlib/collections.py:1448: ComplexWarning: Casting complex values to real discards the imaginary part
  else mpath.Path(np.asarray(seg, float))
/usr/local/lib/python3.10/dist-packages/matplotlib/cbook.py:1345: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


In [123]:
plt.plot(t, [np.real(X(ti))[-1] for ti in t])

xright = np.array([np.real(X(ti))[-1] for ti in t])
plt.plot(t, xright)
t[xright>0.01]

array([ 2.04081633,  2.24489796,  2.44897959,  3.06122449,  3.26530612,
        3.46938776,  3.67346939,  3.87755102,  4.08163265,  4.28571429,
        4.48979592,  5.30612245,  5.51020408,  6.12244898,  6.32653061,
        6.93877551,  7.14285714,  7.55102041,  7.95918367,  8.57142857,
        9.3877551 ,  9.59183673,  9.79591837, 10.        ])

In [72]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

fig, ax = plt.subplots()
x = np.linspace(0.1, 2*np.pi, 50)
y = np.cos(x)
bottom = 0
h_stem = ax.stem(x, y, bottom=bottom,  linefmt='-.')

def update(i):
    y = np.cos(x +  i/50)

    # markerline
    h_stem[0].set_ydata(y)
    h_stem[0].set_xdata(x)  # not necessary for constant x

    # stemlines
    h_stem[1].set_paths([np.array([[xx, bottom],
                                   [xx, yy]]) for (xx, yy) in zip(x, y)])

    # baseline
    h_stem[2].set_xdata([np.min(x), np.max(x)])
    h_stem[2].set_ydata([bottom, bottom])  # not necessary for constant bottom

anim = FuncAnimation(fig, update, frames=range(10, 110, 1), interval=1000)
anim.save('so.gif', dpi=80)
HTML(anim.to_jshtml())

<IPython.core.display.Javascript object>